# Preprocesamiento de la base de datos del clima.

Se mantendrán solo 5 de los 11 climas, el criterio de selección es personal.
La base de datos original se puede acceder desde aqui [Weather image recognition](https://www.kaggle.com/jehanbhathena/weather-dataset?select=dataset).

Los climas utilizados son:
[hail, lighting, rain, sandstorm, snow]

## Librerias

Se importarán las librerias pertinentes para realizar la limpieza y visualización de los datos

- **Numpy**: Librería para manejo de datos y arreglos númericos.
- **Pandas**: Creación de los DataFrame y mejor manejo de los datos.
- **Matplotlib**: Creación de los gráficos y las figuras para visualización de los datos.

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib

In [1]:
# Importamos las librerias principales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Obtenemos el Sistema operativo y definimos el separador de directorios
OS = os.environ.get('OS')
if OS == 'Windows_NT':
    SEPARATOR = '\\'
else:
    SEPARATOR = '/'

%matplotlib inline

## Carga del dataset

Al tener imagenes, la base de datos constará únicamente por el nombre del clima, la ubicación y la cantidad de imagenes.

In [2]:
# Función para filtrar los climas de la base de datos
def load_data(path: str, desired_weather:list=None) -> pd.DataFrame:
    """
    Función para cargar los datos de la base de datos.
    param: path: str, ruta de la base de datos.
    param: desired_weather: list, lista de los climas que se quieren cargar.
    return: df: pd.DataFrame, dataframe con los datos de la base de datos.
    """
    # Asignamos en memoria las variables que contendrán las rutas, nombres y cantidad de imagenes por clima.
    paths = []
    weather_name = []
    num_images = []
    # Obtenemos la ruta completa para evitar algún problema.
    full_path = os.path.realpath(path)

    # Recorremos todos los directorios de la base de datos.
    for root, dirs, _ in os.walk(full_path):
        for dir in dirs:
            # Si el clima se encuentra en la lista se almacena.
            if dir in desired_weather:

                paths.append(os.path.join(root, dir))
                weather_name.append(dir)
                num_images.append(len(os.listdir(os.path.join(root, dir))))

            # Si no se da una lista de climas deseados, se cargarán todos.
            elif desired_weather is None:
                paths.append(os.path.join(root, dir))
                weather_name.append(dir)
                num_images.append(len(os.listdir(os.path.join(root, dir))))

    # Se crea un DataFrame que contiene los datos obtenidos.
    df = pd.DataFrame({'weather': weather_name, 'num_images': num_images, 'path': paths})
    return df


# Definimos la ruta de la base de datos y los climas que queremos cargar
db_path = 'dataset/'
desired_weather = ['hail', 'lightning', 'rain', 'sandstorm', 'snow']

# Obtenemos los datos y mostramos una población del Dataframe.
df = load_data(db_path, desired_weather)
df.head(10)

,weather,num_images,path
0,hail,592,D:\Programación\IA center_proyectofinal\weathe...
1,lightning,378,D:\Programación\IA center_proyectofinal\weathe...
2,rain,527,D:\Programación\IA center_proyectofinal\weathe...
3,sandstorm,692,D:\Programación\IA center_proyectofinal\weathe...
4,snow,621,D:\Programación\IA center_proyectofinal\weathe...


Es necesario tener 300 imagenes minimo por clima para entrenar de mejor manera y tener una cantidad decente de muestras para validación y testeo.

Se crea una columna por clima para ver si les hace falta imagenes o no.

In [3]:
# Obtenemos las filas para añadir la cantidad de imagenes restantes.
TOTAL_IMAGES = 300
filas, columnas = df.shape

data_frame_copy = df.copy()

for i in range(filas):
    # Creamos una columna con las imagenes necesarias para tener las 300 imagenes.
    data_frame_copy.loc[i, 'num_images_needed'] = int(TOTAL_IMAGES - data_frame_copy.loc[i, 'num_images'])

data_frame_copy.drop(columns=['path'], inplace=True)
data_frame_copy.head(10)

,weather,num_images,num_images_needed
0,hail,592,-292.0
1,lightning,378,-78.0
2,rain,527,-227.0
3,sandstorm,692,-392.0
4,snow,621,-321.0


Se puede observar que todas las clases cumplen con el minimo de imagenes deseada (300), sin embargo, se puede observar un desbalanceo de clases que pudiera a llevar un peor aprendizaje de los climas con menos muestras. Esto afecta mas fuertemente a **lightning** y en menor medida a **rain**.

-----------------------------------------
-----------------------------------------

## Preprocesamiento de las imagenes

Se llevará a cabo un preprocesamiento de las imagenes para que todas cuenten con las mismas características. 
Las imagenes se normalizarán con las siguientes características: (64,64,3).

Para esta etapa se utilizarán las siguientes librerias:

- **TensorFlow**: Creación del pipeline de entrenamiento y la creación y entrenamiento del modelo.
- **Scikit-Learn**: Dividir la base de datos y aleatorizar los conjuntos de entrenamiento, pruebas y validación. 
- **Skimage**: Cargar las imagenes y normalizar el tamaño.
- **Joblib**: Almacenamiento de la base de datos en memoria.

In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize

import joblib

Se obtendrán todas las imagenes de la base de datos y se almacenaran en arreglos.

In [26]:
def load_images(path: str, desired_weather: str) -> np.array:

    """
    Función para cargar las imagenes de un clima.
    """ 
    # Obtenemos la ruta completa para evitar algún problema.
    full_path = os.path.realpath(path)

    # Creamos una lista para almacenar las imagenes.
    X = []
    y = []

    # Recorremos todos los directorios de la base de datos.
    for root, dirs, files in os.walk(full_path):
        weather_name = root.split(SEPARATOR)[-1]
        # Obntenemos el nombre del clima
        print('========================')
        print(weather_name)
        if weather_name in desired_weather:
            for file in files:

                image_path = os.path.join(root, file)
                image = imread(image_path)
                image = resize(image, (64, 64, 3))

                X.append(image)
                y.append(weather_name)

                '''
                # Recorremos todas las imagenes del clima.
                for image in os.listdir(os.path.join(root, dir)):
                    # Cargamos la imagen.
                    img = load_img(os.path.join(root, dir, image))
                    # Convertimos la imagen a un array.
                    img = np.array(img)
                    # Añadimos la imagen a la lista.
                    images.append(img)
                '''
            
    return np.array(X), y

# Obtenemos la base de datos con las clases deseadas.
X, y = load_images(db_path, desired_weather)


dataset
dew
fogsmog
frost
glaze
hail
lightning
rain
rainbow
rime
sandstorm
snow


In [31]:
# Observamos que X tenga la dimensión esperada y que y contenga las clases esperadas y la misma cantidad de datos que X
print(X.shape)
print(np.unique(y))
print(len(y))

# Se almacenan los datos en un archivo para no tener que volver a cargarlos.
joblib.dump(X, 'X.joblib')
joblib.dump(y, 'y.joblib')

(2810, 64, 64, 3)
['hail' 'lightning' 'rain' 'sandstorm' 'snow']
2810


['y.joblib']

Obtenemos el conjunto de entrenamiento, pruebas y validación.

Se checa el tamaño del entrenamiento para ver que si sean mas de 200 imagenes.

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=614)
# Checamos que la clase 'lightning tenga el minimo requerido
lightning_size = len([i for i in y_train if i == 'lightning'])
print('El tamaño de entrenamiento de la clase "lightning es de: ', lightning_size)


El tamaño de entrenamiento de la clase "lightning es de:  305


In [ ]:
# Checamos que 